In [1]:
# working to read in the trimmed produce data, and then split it into training and testing sets
# splitting is done using a walk-forward method with a windowed test set of 3 years data incremented in 3 month blocks
# training is always 3 years, test is always 3 months in advance
import pandas as pd
import numpy as np

data_dir = './trimmed_data_and_plots/'

In [2]:
def generate_predict_start_dates(start_date, end_date, train_length, test_length, window_step):
    train_time = pd.to_timedelta(pd.np.ceil(train_length), unit="D")
    test_time = pd.to_timedelta(pd.np.ceil(test_length), unit="D")
    window_time = pd.to_timedelta(pd.np.ceil(window_step), unit='D')
    
    output_dates = [pd.to_datetime(start_date) + train_time]
    most_recent_date = pd.to_datetime(start_date) + train_time
    while most_recent_date + train_time + test_time <= pd.to_datetime(end_date):
        output_dates.append(most_recent_date + window_time)
        most_recent_date = most_recent_date + window_time
    return(output_dates)

In [3]:
def read_trimmed_data(city, veggie):
    # reads in cleaned .csv data for one city and veggie, returns
    output_data = pd.read_csv(data_dir+veggie+'_'+city+'_TRIM.csv')
    output_data['Date'] = pd.to_datetime(output_data['Date'])
    output_data = output_data.sort_values(by='Date')
    output_data = output_data.reset_index(drop=True)
    return output_data

In [4]:
def nearest_date(dates, targdate):
    for i in dates:
        i = i.to_pydatetime()
    nearest = min(dates, key=lambda x: abs(x - targdate))
    timedelta = abs(nearest - targdate)
    return nearest, timedelta

In [5]:
def get_train_test(veggie_data, start_date, train_days, predict_days):
    all_dates = pd.to_datetime(veggie_data['Date'])
    train_time = pd.to_timedelta(pd.np.ceil(train_days), unit="D")
    test_time = pd.to_timedelta(pd.np.ceil(predict_days), unit="D")
    start_date_train = start_date - train_time
    nearest_date_train, deltatrain = nearest_date(all_dates, start_date_train)
    end_date_test = start_date + test_time
    nearest_date_test, deltatest = nearest_date(all_dates, end_date_test)
    training_set = veggie_data[(veggie_data['Date'] >= nearest_date_train) & (veggie_data['Date'] < start_date)]
    test_set = veggie_data[(veggie_data['Date'] >= start_date) & (veggie_data['Date'] <= nearest_date_test)]
    return(training_set, test_set)

In [6]:
def naive_historical_average(veggie_data, start_dates, train_days, predict_days):
    train_time = pd.to_timedelta(pd.np.ceil(train_days), unit="D")
    test_time = pd.to_timedelta(pd.np.ceil(predict_days), unit="D")
    one_year = pd.to_timedelta(pd.np.ceil(365), unit='D')
    # first compute the historical average values
    predictions = []
    actuals = []
    for i in start_dates:
        current_train_data, current_test_data = get_train_test(veggie_data, i, train_days, predict_days)
        # at some point i'll want to make this more intelligent and handle missing data
        actuals.append(current_test_data['Average Price'].median())
        # now do the naive historical average model
        num_years = train_days // 365
        historical_medians = []
        for y in range(num_years):
            historical_data = veggie_data[(veggie_data['Date'] >= (i-(y+1)*one_year)) & (veggie_data['Date'] <= (i-(y+1)*one_year + test_time))]
            historical_medians.append(historical_data['Average Price'].median())
        predictions.append(pd.Series(historical_medians).median())
    return predictions, actuals


In [7]:
# define some variables
cities = ['NEW+YORK']
veggies = ['APPLES','APRICOTS','ASPARAGUS','AVOCADOS','BANANAS','BEANS','BEETS','BLACKBERRIES','BLUEBERRIES','BROCCOLI','BRUSSELS+SPROUTS','CABBAGE','CANTALOUPS','CARROTS','CAULIFLOWER','CELERY','CHERRIES','CLEMENTINES', 'CUCUMBERS','ENDIVE','GARLIC','GINGER+ROOT','GRAPEFRUIT','GRAPES','HONEYDEWS','KIWIFRUIT','LEMONS','LETTUCE%2C+ICEBERG','LETTUCE%2C+ROMAINE','LETTUCE%2C+RED+LEAF','LETTUCE%2C+GREEN+LEAF', 'LIMES','MANGOES','NECTARINES','OKRA','ORANGES','PEACHES','PEARS','PEAS+GREEN','PEPPERS%2C+BELL+TYPE','PINEAPPLES','PLUMS','POTATOES','RADISHES','RASPBERRIES','RHUBARB','SPINACH','SQUASH','STRAWBERRIES','TURNIPS']

master_start_date = '20070601'
master_end_date = '20170531'
# three year train, 3 month test
train_length = 365*3
predict_length = 90
# increment window of train/test before generating new set
increment_test = 115

In [27]:
datelist = generate_predict_start_dates(master_start_date, master_end_date, train_length, predict_length, increment_test)

output_naive = pd.DataFrame(datelist).rename(index=int, columns={0: "Date"})

labels = ['Item', 'PCT Miss']
overall_stats_naive = []

for c in cities:
    for v in veggies:
        current_veggie = read_trimmed_data(c, v)
        pred, act = naive_historical_average(current_veggie, datelist, train_length, predict_length)
        output_naive = pd.concat([output_naive, pd.Series(pred).rename(v+'_PRED'), pd.Series(act).rename(v+'_ACT')], axis=1)
        pct_miss = abs((output_naive[v + '_ACT'] - output_naive[v + '_PRED'])/output_naive[v + '_ACT']).median()
        overall_stats_naive.append((v, pct_miss))

summary_data_naive = pd.DataFrame.from_records(overall_stats_naive, columns=labels)

summary_data_naive.to_csv('naive_model_output')

,Item,PCT Miss
0,APRICOTS,0.156757


In [ ]:
# now that's done, compute the median percent misses for each fruit
